# Chapter 3 - Classification (分类)

_This notebook contains all the sample code and solutions to the exercises in chapter 3_

# Setup
First, let's make sure this notebook works well in both python 2 and python 3, import a new common modules, ensure MatplotLib plots figures inline and prepare a funciton to save the figures:

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [ ]:
def sort_by_target(mnist):
    reorder_train = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[:60000])]))[:,1]
    reorder_test = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[60000:])]))[:, 1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000] = mnist.target[reorder_train]
    mnist.data[60000:] = mnist.data[reorder_test + 60000]
    mnist.target[60000:] = mnist.target[reorder_test + 60000]

try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True)
    mnist.target = mnist.target.astype(np.int8)
    sort_by_target(mnist)
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata("MNIST original")

X, y = mnist["data"], mnist["target"]


In [ ]:
X.shape, y.shape

In [ ]:
# 显示单个图片
def plot_digit(data):
    image = data.reshape(28,28)
    plt.imshow(image, cmap=mpl.cm.binary, interpolation="nearest")
    plt.axis("off")


In [ ]:
some_digit = X[30000]
plot_digit(some_digit)
print(y[30000])

In [ ]:
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(9,9))
example_images = np.r_[X[:12000:600],X[13000:30600:600],X[30600:60000:590]]
plot_digits(example_images, images_per_row=10)
plt.show()

***对数据初步探索后，在深入研究这些数据之前，先创建测试集***

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


In [ ]:
# 给数据洗牌
import numpy as np

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]


In [ ]:
# 先训练一个二元分类器，简化问题，比如只识别4和非4(教程是5，但数据显示是4)
# 创建目标向量
y_train_4 = (y_train == 4)
y_test_4 = (y_test == 4)

# 选择一个分类器并开始训练，一个好的选择是使用SGD（随机梯度下降）分类器
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_4)

In [ ]:
# 检测
sgd_clf.predict([some_digit])

In [ ]:
# 评估模型性能
# 实施交叉验证
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_4, cv=3, scoring="accuracy")

In [ ]:
# 使用StratifiedKFold，与上面的cross_val_score一致
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42)
for train_index, test_index in skfolds.split(X_train, y_train_4):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = (y_train_4[train_index])
    X_test_fold = X_train[test_index]
    y_test_fold = (y_train_4[test_index])

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct/len(y_pred))


In [ ]:
# 一个更简单的分类器，预测出来的准确率也达到了90%+，说明使用准确率无法作为分类器的首选性能指标
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X),1), dtype=bool)

never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_4, cv=3, scoring="accuracy")



In [ ]:
# 评估分类器的更好方法是：混淆矩阵
# 总体思路就是：统计A类别实例 被 分成 B类别的次数
# 要计算混淆矩阵，首先需要有一组预测才能将其与实际目标进行比较。可以使用cross_val_predict函数
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_4, cv=3)

# 使用confusion_matrix()函数来获取混淆矩阵；行表示实际类别，列表示预测类别
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_4, y_train_pred)

**行表示实际类别，列表示预测类别**

实际非4\[53633,   525], 有53633张非4被正确预测，有525非4被错误的预测成了4（假负类）

  实际4\[  998,  4844]  有998张4被预测成了非4，有4844张4被正确预测为4(真正类)

  其中：FN=53633，FP=525，TN=998，TP=4844

  精度 = TP/(TP+FP)  __正类__预测的准确率；(在所有判断为恐怖分子中，真正的恐怖分子比例)

  召回率 = TP/(TP+FN)  __分类器正确检测__到的正类实例的比例（也就是正确判为恐怖分子占实际所有恐怖分子的比例）